In [8]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

import numpy as np
import pandas as pd

from tune_sklearn import TuneGridSearchCV 

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso, LassoCV, ElasticNet
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score,RandomizedSearchCV
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

ModuleNotFoundError: No module named 'tune_sklearn'

In [4]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x_train.values)
    rmse= np.sqrt(-cross_val_score(model, x_train.values, y_train.values, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models

    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]

        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self

    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

NameError: name 'BaseEstimator' is not defined

In [5]:
def rmsle_cv2(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x_train_all.values)
    rmse= np.sqrt(-cross_val_score(model, x_train_all.values, y_train_all.values, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)



Fitting 5 folds for each of 100 candidates, totalling 500 fits


{'n_estimators': 50,
 'min_samples_split': 6,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 40,
 'bootstrap': False}

RandomForestRegressor(bootstrap=False, max_depth=40, max_features='sqrt',
                      min_samples_split=6, n_estimators=50)

rf score: 0.1338 (0.0071)


In [4]:
x_train = pd.read_csv('X_train.csv')
x_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')


x_train_all=pd.read_csv('xtrain_all.csv')
y_train_all=pd.read_csv('ytrain_all.csv')

In [5]:

x_train=x_train.drop('Id',axis=1)
x_train.head()

,MSSubClass,LotFrontage,LotArea,Alley,LotShape,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,2.885846,5.831328,19.212182,0.730463,1.540963,0.0,2.440268,1.820334,14.187527,14.187527,...,0,0,0,1,0,0,0,0,1,0
1,2.055642,6.221214,19.712205,0.730463,1.540963,0.0,2.259674,2.440268,14.145138,14.145138,...,0,0,0,1,0,0,0,0,1,0
2,2.885846,5.914940,20.347241,0.730463,0.000000,0.0,2.440268,1.820334,14.184404,14.185966,...,0,0,0,1,0,0,0,0,1,0
3,3.011340,5.684507,19.691553,0.730463,0.000000,0.0,2.440268,1.820334,14.047529,14.135652,...,0,0,0,1,1,0,0,0,0,0
4,2.885846,6.314735,21.325160,0.730463,0.000000,0.0,2.602594,1.820334,14.182841,14.182841,...,0,0,0,1,0,0,0,0,1,0


In [68]:
# Lasso 5-fold cv on original data
model = LassoCV(cv=5, random_state=1, max_iter=10000)
model.fit(x_train, y_train)
LassoCV(cv=5, max_iter=10000, random_state=1)
model.alpha_

0.0006952886354228365

In [9]:
# Lasso 5-fold cv on synthetic+oringioal data
model2 = LassoCV(cv=5, random_state=1, max_iter=10000)
model2.fit(x_train_all, y_train_all)
LassoCV(cv=5, max_iter=10000, random_state=1)
model2.alpha_

0.00391011042423136

In [69]:
lasso_parm = model.alpha_
lasso = make_pipeline(RobustScaler(), Lasso(alpha =model.alpha_, random_state=1))

In [11]:
lasso_parm = model2.alpha_
lasso2 = make_pipeline(RobustScaler(), Lasso(alpha =model2.alpha_, random_state=1))

In [30]:
score = rmsle_cv2(lasso2)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso score: 0.3101 (0.1816)


In [70]:
# Use grid search to tune the parameters:
parametersGrid = {"max_iter": [1, 5, 10],
                  "alpha": np.arange(0.001, 0.01, 0.001),
                  "l1_ratio": np.arange(0.0, 1.0, 0.5),
                  "random_state": np.arange(1, 3, 1)}

eNet = ElasticNet()
grid = GridSearchCV(eNet, parametersGrid, scoring='accuracy', cv=10)
grid2 = GridSearchCV(eNet, parametersGrid, scoring='accuracy', cv=10)
grid.fit(x_train, y_train)
grid2.fit(x_train_all, y_train_all)

GridSearchCV(cv=10, estimator=ElasticNet(),
             param_grid={'alpha': array([0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009]),
                         'l1_ratio': array([0. , 0.5]), 'max_iter': [1, 5, 10],
                         'random_state': array([1, 2])},
             scoring='accuracy')

In [71]:

parms = grid.best_params_
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=parms['alpha'], l1_ratio=parms['l1_ratio'], random_state=parms['random_state']))


parms2 = grid2.best_params_
ENet2 = make_pipeline(RobustScaler(), ElasticNet(alpha=parms2['alpha'], l1_ratio=parms2['l1_ratio'], random_state=parms2['random_state']))

print("enet parm1 :",parms)
print("enet parm2 :",parms2)

enet parm1 : {'alpha': 0.001, 'l1_ratio': 0.0, 'max_iter': 1, 'random_state': 1}
enet parm2 : {'alpha': 0.001, 'l1_ratio': 0.0, 'max_iter': 1, 'random_state': 1}


In [72]:
score = rmsle_cv(ENet)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv2(ENet2)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso score: 0.1172 (0.0092)

Lasso score: 0.3307 (0.1979)


In [6]:
ridge = Ridge()
param_grid = {
    'alpha': np.arange(0.1, 3, 0.1)
}
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid)
grid_search.fit(x_train, y_train)

GridSearchCV(estimator=Ridge(),
             param_grid={'alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9])})

In [7]:
ridge = Ridge()
param_grid = {
    'alpha': np.arange(0.1, 3, 0.1)
}
grid_search2 = GridSearchCV(estimator=ridge, param_grid=param_grid)
grid_search2.fit(x_train_all, y_train_all)

GridSearchCV(estimator=Ridge(),
             param_grid={'alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9])})

In [8]:
Ridge_parm = grid_search.best_params_
ridge = Ridge(grid_search.best_params_['alpha'])

Ridge_parm2 = grid_search2.best_params_
ridge2 = Ridge(grid_search2.best_params_['alpha'])

In [10]:
grid_search.best_params_
grid_search.best_params_['alpha']

2.9000000000000004

In [76]:
score = rmsle_cv(ridge)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv2(ridge2)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso score: 0.1158 (0.0086)

Lasso score: 0.3290 (0.1964)


In [77]:
n_estimators = [5,20,50,100] 
max_features = ['auto', 'sqrt'] 
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] 
min_samples_split = [2, 6, 10]
min_samples_leaf = [1, 3, 4] 
bootstrap = [True, False] 

random_grid2 = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap}

print(random_grid2)

{'n_estimators': [5, 20, 50, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120], 'min_samples_split': [2, 6, 10], 'min_samples_leaf': [1, 3, 4], 'bootstrap': [True, False]}


In [78]:
rf = RandomForestRegressor()

rf_random =RandomizedSearchCV(estimator = rf, param_distributions = random_grid2,n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train, y_train)


rf_random.best_params_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


{'n_estimators': 50,
 'min_samples_split': 6,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 100,
 'bootstrap': False}

In [42]:
parms1=rf_random.best_params_
rfmod = RandomForestRegressor(n_estimators = parms1['n_estimators'] , min_samples_split = parms1['min_samples_split'], min_samples_leaf= parms1['min_samples_leaf'], max_features = parms1['max_features'], max_depth= parms1['max_depth'], bootstrap=parms1['bootstrap']) 
rfmod.fit(x_train, y_train) 

RandomForestRegressor(bootstrap=False, max_depth=40, max_features='sqrt',
                      min_samples_split=6, n_estimators=50)

In [43]:
rf = RandomForestRegressor()

rf_random2 =RandomizedSearchCV(estimator = rf, param_distributions = random_grid2,n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)
# Fit the random search model
rf_random2.fit(x_train_all, y_train_all)


rf_random2.best_params_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


{'n_estimators': 100,
 'min_samples_split': 6,
 'min_samples_leaf': 3,
 'max_features': 'sqrt',
 'max_depth': 40,
 'bootstrap': False}

In [44]:
parms2=rf_random2.best_params_
rfmod2 = RandomForestRegressor(n_estimators = parms2['n_estimators'] , min_samples_split = parms2['min_samples_split'], min_samples_leaf= parms2['min_samples_leaf'], max_features = parms2['max_features'], max_depth= parms2['max_depth'], bootstrap=parms2['bootstrap']) 
rfmod2.fit(x_train_all, y_train_all) 

RandomForestRegressor(bootstrap=False, max_depth=40, max_features='sqrt',
                      min_samples_leaf=3, min_samples_split=6)

In [79]:
score = rmsle_cv(rfmod)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv2(rfmod2)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso score: 0.1332 (0.0074)

Lasso score: 0.2893 (0.1726)


In [80]:
gbr = GradientBoostingRegressor()
parameters = {
    'n_estimators': np.arange(2000, 4000, 500),
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': np.arange(1, 5, 1)
}

grid_search = TuneGridSearchCV(gbr, parameters)
grid_search.fit(x_train, y_train)

(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(rayl

(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(rayl

(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(rayl

(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(_Trainable pid=27996) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 2x across cluster]
(_Trainable pid=27996)   y = column_or_1d(y, warn=True) [repeated 2x across cluster]
(_Trainable pid=28516) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarnin

(_Trainable pid=28464) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 3x across cluster]
(_Trainable pid=28464)   y = column_or_1d(y, warn=True) [repeated 3x across cluster]
2023-05-05 19:18:54,906	INFO tensorboardx.py:269 -- Removed the following hyperparameter values when logging to tensorboard: {'max_depth': ('__ref_ph', '0ab215e5'), 'n_estimators': ('__ref_ph', 'fe0afe46')}
2023-05-05 19:18:55,507	INFO tensorboardx.py:269 -- Removed the following hyperparameter values when logging to tensorboard: {'max_depth': ('__ref_ph', '0ab215e5'), 'n_estimators': ('__ref_ph', 'fe0afe46')}
(_Trainable pid=27920) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, 

2023-05-05 19:20:28,049	INFO tensorboardx.py:269 -- Removed the following hyperparameter values when logging to tensorboard: {'max_depth': ('__ref_ph', 'b5858eb6'), 'n_estimators': ('__ref_ph', 'fe0afe46')}
2023-05-05 19:20:28,438	INFO tensorboardx.py:269 -- Removed the following hyperparameter values when logging to tensorboard: {'max_depth': ('__ref_ph', 'b5858eb6'), 'n_estimators': ('__ref_ph', 'fe0afe46')}
(_Trainable pid=28272) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 6x across cluster]
(_Trainable pid=28272)   y = column_or_1d(y, warn=True) [repeated 6x across cluster]
(_Trainable pid=28372) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, 

2023-05-05 19:22:21,492	INFO tensorboardx.py:269 -- Removed the following hyperparameter values when logging to tensorboard: {'max_depth': ('__ref_ph', 'b5858eb6'), 'n_estimators': ('__ref_ph', '6b04a990')}
(_Trainable pid=28272) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 5x across cluster]
(_Trainable pid=28272)   y = column_or_1d(y, warn=True) [repeated 5x across cluster]
(_Trainable pid=27920) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 2x across cluster]
(_Trainable pid=27920)   y = column_or_1d(y, warn=True) [repeated 2x across cluster]
(_Trainable pid=27804) C:\ProgramData\Anaconda3\lib\site-packag

(_Trainable pid=28516)   y = column_or_1d(y, warn=True)
(_Trainable pid=28516)   y = column_or_1d(y, warn=True)
(_Trainable pid=27916) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
(_Trainable pid=27916)   y = column_or_1d(y, warn=True)
(_Trainable pid=28464) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 2x across cluster]
(_Trainable pid=28464)   y = column_or_1d(y, warn=True) [repeated 2x across cluster]
2023-05-05 19:24:47,513	INFO tensorboardx.py:269 -- Removed the following hyperparameter values when logging to tensorboard: {'max_depth': ('__ref_ph', '0ab215e5'), 'n_estimators': ('__ref_ph', '0e32b97d')}
2023-05-0

TuneGridSearchCV(estimator=GradientBoostingRegressor(), mode='max', n_jobs=-1,
                 param_grid={'learning_rate': [0.1, 0.05, 0.01],
                             'max_depth': array([1, 2, 3, 4]),
                             'n_estimators': array([2000, 2500, 3000, 3500])},
                 sk_n_jobs=1)

In [81]:
GB_parms = grid_search.best_params_
parms = grid_search.best_params_
GBoost = GradientBoostingRegressor(n_estimators=parms['n_estimators'], learning_rate=parms['learning_rate'],
                                   max_depth=parms['max_depth'])


In [48]:
gbr = GradientBoostingRegressor()
parameters = {
    'n_estimators': np.arange(2000, 4000, 500),
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': np.arange(1, 5, 1)
}
grid_search2 = TuneGridSearchCV(gbr, parameters)
grid_search2.fit(x_train_all, y_train_all)

(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(rayl

(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(rayl

(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\

(raylet) Error processing line 1 of C:\ProgramData\Anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 562, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(_Trainable pid=14112) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 3x across cluster]
(_Trainable pid=14112)   y = column_or_1d(y, warn=True) [repeated 3x across cluster]
(_Trainable pid=6876) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning

(_Trainable pid=8600) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 5x across cluster]
(_Trainable pid=8600)   y = column_or_1d(y, warn=True) [repeated 5x across cluster]
(_Trainable pid=23544) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 2x across cluster]
(_Trainable pid=23544)   y = column_or_1d(y, warn=True) [repeated 2x across cluster]
(_Trainable pid=4868)   y = column_or_1d(y, warn=True)
(_Trainable pid=4868)   y = column_or_1d(y, warn=True)
(_Trainable pid=18068) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array

(_Trainable pid=4864) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 2x across cluster]
(_Trainable pid=4864)   y = column_or_1d(y, warn=True) [repeated 2x across cluster]
(_Trainable pid=8600) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 2x across cluster]
(_Trainable pid=8600)   y = column_or_1d(y, warn=True) [repeated 2x across cluster]
(_Trainable pid=18068) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 3x across clu

2023-05-05 17:53:27,526	INFO tensorboardx.py:269 -- Removed the following hyperparameter values when logging to tensorboard: {'max_depth': ('__ref_ph', '500726e3'), 'n_estimators': ('__ref_ph', '6b04a990')}
(_Trainable pid=23544)   y = column_or_1d(y, warn=True)
(_Trainable pid=23544)   y = column_or_1d(y, warn=True)
(_Trainable pid=9772) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
(_Trainable pid=9772)   y = column_or_1d(y, warn=True)
(_Trainable pid=18068) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 2x across cluster]
(_Trainable pid=18068)   y = column_or_1d(y, warn=True) [repeated 2x across cluster]
(_Trainable

(_Trainable pid=18068) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 2x across cluster]
(_Trainable pid=18068)   y = column_or_1d(y, warn=True) [repeated 2x across cluster]
(_Trainable pid=8248) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 3x across cluster]
(_Trainable pid=8248)   y = column_or_1d(y, warn=True) [repeated 3x across cluster]
(_Trainable pid=4864)   y = column_or_1d(y, warn=True)
(_Trainable pid=4864)   y = column_or_1d(y, warn=True)
2023-05-05 17:57:52,198	INFO tensorboardx.py:269 -- Removed the following hyperparameter values when logging to tensorboard: {'max_depth': ('__ref_ph', '0ab215e5'

2023-05-05 18:02:23,797	INFO tensorboardx.py:269 -- Removed the following hyperparameter values when logging to tensorboard: {'max_depth': ('__ref_ph', '0ab215e5'), 'n_estimators': ('__ref_ph', '0e32b97d')}
(_Trainable pid=8600)   y = column_or_1d(y, warn=True)
(_Trainable pid=8600)   y = column_or_1d(y, warn=True)
(_Trainable pid=8292) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
(_Trainable pid=8292)   y = column_or_1d(y, warn=True)
(_Trainable pid=14112) C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). [repeated 2x across cluster]
(_Trainable pid=14112)   y = column_or_1d(y, warn=True) [repeated 2x across cluster]
(_Trainable p

TuneGridSearchCV(estimator=GradientBoostingRegressor(), mode='max', n_jobs=-1,
                 param_grid={'learning_rate': [0.1, 0.05, 0.01],
                             'max_depth': array([1, 2, 3, 4]),
                             'n_estimators': array([2000, 2500, 3000, 3500])},
                 sk_n_jobs=1)

In [49]:
GB_parms2 = grid_search2.best_params_
parms2 = grid_search2.best_params_
GBoost2 = GradientBoostingRegressor(n_estimators=parms2['n_estimators'], learning_rate=parms2['learning_rate'],
                                   max_depth=parms2['max_depth'])


In [82]:
score = rmsle_cv(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(ridge)
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(rfmod)
print("rf score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
averaged_models = AveragingModels(models = (ENet, GBoost, ridge, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso score: 0.1107 (0.0075)

Ridge score: 0.1158 (0.0086)

ElasticNet score: 0.1172 (0.0092)

Gradient Boosting score: 0.1168 (0.0061)

rf score: 0.1330 (0.0066)

 Averaged base models score: 0.1096 (0.0076)


In [51]:
score = rmsle_cv2(lasso2)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv2(ridge2)
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv2(ENet2)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv2(GBoost2)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv2(rfmod2)
print("rf score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
averaged_models2 = AveragingModels(models = (ENet2, GBoost2, ridge2, lasso2))

score = rmsle_cv2(averaged_models2)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso score: 0.3101 (0.1816)

Ridge score: 0.3290 (0.1964)

ElasticNet score: 0.3307 (0.1979)

Gradient Boosting score: 0.2922 (0.1816)

rf score: 0.2889 (0.1714)

 Averaged base models score: 0.3037 (0.1874)


In [18]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [19]:
averaged_models.fit(x_train.values, y_train.values)
stacked_train_pred = averaged_models.predict(x_train.values)
stacked_pred = np.expm1(averaged_models.predict(x_test.values))
print(rmsle(y_train.values, stacked_train_pred))

0.08928750138956935


In [43]:
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
sub = pd.DataFrame()
sub['Id'] = test_df_Id
sub['SalePrice'] = stacked_pred
sub.to_csv('submission.csv',index=False)

In [17]:
print(lasso_parm)
print(ENet_parm)
print(GB_parms)
print(Ridge_parm)

0.0006952886354228365
{'alpha': 0.001, 'l1_ratio': 0.0, 'max_iter': 1, 'random_state': 1}
{'n_estimators': 2500, 'learning_rate': 0.01, 'max_depth': 2}
{'alpha': 2.9000000000000004}


In [5]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0002, random_state=0))
ridge = Ridge(0.008)
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0002, random_state=0))
score = rmsle_cv(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(ridge)
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso score: 0.1122 (0.0076)

Ridge score: 0.1233 (0.0104)


In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=parms['alpha'], l1_ratio=parms['l1_ratio'], random_state=parms['random_state']))

In [7]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0003248385))
score = rmsle_cv(ENet)
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Ridge score: 0.1128 (0.0077)


In [5]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor

# Create the individual models
ridge = Ridge(2.9000000000000004)
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0006952886354228365, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.001, l1_ratio=0, random_state=1))
GBoost = GradientBoostingRegressor(n_estimators=2500, learning_rate=0.01,
                                   max_depth = 2)

# Create the ensemble model
ensemble = VotingRegressor(
    estimators=[('ridge', ridge), ('lasso', lasso), ('elastic_net', ENet), ('gradient_boosting', GBoost)]
)

# Fit the ensemble model on the training data
ensemble.fit(x_train, y_train)

# Make predictions on new data
predictions = np.expm1(ensemble.predict(x_test))

In [6]:
score = rmsle_cv(ensemble)
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Ridge score: 0.1100 (0.0076)


In [9]:
predictions = np.expm1(ensemble.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
sub = pd.DataFrame()
sub['Id'] = test_df_Id
sub['SalePrice'] = predictions
sub.to_csv('submission.csv',index=False)

In [10]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
meta_model = LinearRegression()
ensemble = StackingRegressor(
    estimators=[('ridge', ridge), ('lasso', lasso), ('elastic_net', ENet), ('gradient_boosting', GBoost)],
    final_estimator=meta_model
)

score = rmsle_cv(ensemble)
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Ridge score: 0.1090 (0.0070)


In [11]:
ensemble.fit(x_train, y_train)
predictions = np.expm1(ensemble.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
sub = pd.DataFrame()
sub['Id'] = test_df_Id
sub['SalePrice'] = predictions
sub.to_csv('submission.csv',index=False)

In [12]:
model = GradientBoostingRegressor()
score = rmsle_cv(model)
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Ridge score: 0.1202 (0.0057)


In [13]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
meta_model = LinearRegression()
ensemble = StackingRegressor(
    estimators=[('ridge', ridge), ('lasso', lasso), ('elastic_net', ENet)],
    final_estimator=GBoost
)

ensemble.fit(x_train, y_train)
predictions = np.expm1(ensemble.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
sub = pd.DataFrame()
sub['Id'] = test_df_Id
sub['SalePrice'] = predictions
sub.to_csv('submission.csv',index=False)

In [85]:
x_test.head()
x_test=x_test.drop('Id',axis=1)

In [86]:
x_test.head()

,MSSubClass,LotFrontage,LotArea,Alley,LotShape,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,2.055642,6.221214,20.479373,0.730463,1.540963,0.0,2.055642,2.055642,14.121376,14.121376,...,0,0,0,1,0,0,0,0,1,0
1,2.055642,6.244956,21.327220,0.730463,0.000000,0.0,2.259674,2.055642,14.116605,14.116605,...,0,0,0,1,0,0,0,0,1,0
2,2.885846,6.073289,21.196905,0.730463,0.000000,0.0,2.055642,1.820334,14.178149,14.179714,...,0,0,0,1,0,0,0,0,1,0
3,2.885846,6.172972,19.865444,0.730463,0.000000,0.0,2.259674,2.055642,14.179714,14.179714,...,0,0,0,1,0,0,0,0,1,0
4,0.000000,5.093857,17.257255,0.730463,0.000000,0.0,2.602594,1.820334,14.170316,14.170316,...,0,0,0,1,0,0,0,0,1,0


In [60]:
# test set sub for gboost2 on synthetic data 
GBoost2.fit(x_train_all, y_train_all)
predictionsgb2 = np.expm1(GBoost2.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
subgb2 = pd.DataFrame()
subgb2['Id'] = test_df_Id
subgb2['SalePrice'] = predictionsgb2
subgb2.to_csv('submission_gb2.csv',index=False)

In [91]:
#gboost1
GBoost.fit(x_train, y_train)
predictionsgb = np.expm1(GBoost.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
subgb = pd.DataFrame()
subgb['Id'] = test_df_Id
subgb['SalePrice'] = predictionsgb
subgb.to_csv('submission_gb1.csv',index=False)

In [61]:
#test set sub for rfmod2
rfmod2.fit(x_train_all, y_train_all)
predictionsrf2 = np.expm1(rfmod2.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
subrf2 = pd.DataFrame()
subrf2['Id'] = test_df_Id
subrf2['SalePrice'] = predictionsrf2
subrf2.to_csv('submission_rf2.csv',index=False)

In [87]:
#test set sub for rfmod1
rfmod.fit(x_train, y_train)
predictionsrf = np.expm1(rfmod.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
subrf = pd.DataFrame()
subrf['Id'] = test_df_Id
subrf['SalePrice'] = predictionsrf
subrf.to_csv('submission_rfcorrect1.csv',index=False)

In [88]:
#avg2 
averaged_models2.fit(x_train_all, y_train_all)
predictionsavg2 = np.expm1(averaged_models2.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
subavg2 = pd.DataFrame()
subavg2['Id'] = test_df_Id
subavg2['SalePrice'] = predictionsavg2
subavg2.to_csv('submission_avg2.csv',index=False)

In [90]:
#avg1
averaged_models.fit(x_train, y_train)
predictionsavg1 = np.expm1(averaged_models.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
subavg1 = pd.DataFrame()
subavg1['Id'] = test_df_Id
subavg1['SalePrice'] = predictionsavg1
subavg1.to_csv('submission_avg1.csv',index=False)

In [92]:
#lasso1 finished 
lasso.fit(x_train, y_train)
predictionsla1 = np.expm1(lasso.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
subla1 = pd.DataFrame()
subla1['Id'] = test_df_Id
subla1['SalePrice'] = predictionsla1
subla1.to_csv('submission_lasso1.csv',index=False)

In [94]:
#lasso2 finished 
lasso2.fit(x_train_all, y_train_all)
predictionsla2 = np.expm1(lasso2.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
subla2 = pd.DataFrame()
subla2['Id'] = test_df_Id
subla2['SalePrice'] = predictionsla2
subla2.to_csv('submission_lasso2.csv',index=False)

In [95]:
#ridg1 finished 
ridge.fit(x_train, y_train)
predictionsri = np.expm1(ridge.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
subri = pd.DataFrame()
subri['Id'] = test_df_Id
subri['SalePrice'] = predictionsri
subri.to_csv('submission_ridge.csv',index=False)

In [97]:
#ridg2 finished 
ridge2.fit(x_train_all, y_train_all)
predictionsri2 = np.expm1(ridge2.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
subri2 = pd.DataFrame()
subri2['Id'] = test_df_Id
subri2['SalePrice'] = predictionsri2
subri2.to_csv('submission_ridge2.csv',index=False)

In [99]:
#e1 finished 
ENet.fit(x_train, y_train)
predictionsen = np.expm1(ENet.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
suben = pd.DataFrame()
suben['Id'] = test_df_Id
suben['SalePrice'] = predictionsen
suben.to_csv('submission_en.csv',index=False)

In [100]:
#e1 finished 
ENet2.fit(x_train_all, y_train_all)
predictionsen2 = np.expm1(ENet2.predict(x_test))
test_df = pd.read_csv('test.csv')
test_df_Id = test_df['Id']
suben2 = pd.DataFrame()
suben2['Id'] = test_df_Id
suben2['SalePrice'] = predictionsen2
suben2.to_csv('submission_en2.csv',index=False)